In [ ]:
import os
import cv2
import time

from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Egor\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Загрузка предобученной модели YOLOv10 (можно 'yolov10n.pt', 'yolov10s.pt' или yolov10m.pt)
# model = YOLO('yolov10n.pt')  # nano-версия (быстрая, но менее точная)

model = YOLO('yolov10m.pt')  # рекомендуемая модель к обучению

# Обучение модели
results = model.train(data='dataset/data.yaml', epochs=100, imgsz=640, model='yolov10m.pt')

KeyboardInterrupt: 

# подумать с моделью!

In [ ]:
model = YOLO('yolov8s-seg.pt')  # или yolov8n-seg.pt для быстрого старта
model.train(data='data.yaml', epochs=100, imgsz=640, batch=16,
            augment=True, lr0=0.01, patience=50, project='runs/exp_passengers')


In [ ]:
# Загружаем предобученную модель YOLOv10m
model = YOLO('yolov10m.pt')  # уже обучена на COCO

# Тренировка (fine-tuning)
model.train(
    data='data.yaml',       # путь к твоему датасету
    epochs=150,             # побольше эпох для дообучения
    imgsz=1280,             # больше размер — лучше детект людей
    batch=16,               # уменьшить, если мало памяти
    optimizer="AdamW",      # лучше для стабильности
    lr0=0.002,              # чуть меньше, чтобы не "затирать" предобученные веса
    patience=50,            # ранняя остановка, если метрика не растёт
    augment=True,           # включаем аугментации !!!!
    close_mosaic=15,        # отключаем mosaic ближе к концу
    project='runs/passengers',
    name='yolov8m-passengers',
    pretrained=True         # дообучаем предобученные веса
)


In [1]:
import torch
print(torch.__version__)          # Должно быть: 2.x.x+cuXXX (например, 2.3.0+cu118)
print(torch.cuda.is_available())  # Должно быть True

2.5.1+cu121
True


In [2]:
# train.py
from ultralytics import YOLO
import yaml
import os
from pathlib import Path

def clean_dataset(data_yaml_path):
    """Удаляет изображения без разметки из датасета"""
    with open(data_yaml_path) as f:
        data = yaml.safe_load(f)
    
    # Пути к папкам с изображениями и разметкой
    train_images = Path(data['train'])
    val_images = Path(data['val'])
    train_labels = Path(data['train'].replace('images', 'labels'))
    val_labels = Path(data['val'].replace('images', 'labels'))
    
    def clean_folder(images_dir, labels_dir):
        deleted = 0
        for img_path in images_dir.glob('*.*'):
            label_path = labels_dir / f"{img_path.stem}.txt"
            if not label_path.exists():
                img_path.unlink()
                deleted += 1
        return deleted
    
    # Очистка train и val
    train_deleted = clean_folder(train_images, train_labels)
    val_deleted = clean_folder(val_images, val_labels)
    
    print(f"Удалено {train_deleted} train и {val_deleted} val изображений без разметки")

# Проверка CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

# Очистка датасета перед обучением
clean_dataset('data.yaml')

# Устройство
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Используется: {device}")

# Загрузка модели
model = YOLO('yolov10m.pt').to(device)

# Обучение с оптимизированными параметрами
model.train(
    data='data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,
    device=device,
    name='yolov10n_bus_stop_finetuned',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train',
    workers=4,  # Оптимальное количество для загрузки данных
    cache=True,  # Кэширование для ускорения
    amp=True     # Mixed Precision Training
)

# Сохранение модели
model.save('models/yolov10n_best.pt')
print("✅ Модель дообучена и сохранена.")

PyTorch version: 2.5.1+cu121
CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Удалено 0 train и 0 val изображений без разметки
Используется: 0
Ultralytics 8.3.177  Python-3.12.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10m.pt, momentum=0.937, mosaic=1.0, mul

train: Scanning C:\Users\Egor\Desktop\ss\AI-Flow-Detecting\ai-core\dataset\train_2\obj_Train_data.cache... 394 images, 338 backgrounds, 0 corrupt: 100%|██████████| 394/394 [00:00<?, ?it/s]


WARNING cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.3GB RAM): 100%|██████████| 394/394 [00:00<00:00, 402.83it/s]


val: Fast image access  (ping: 0.20.1 ms, read: 421.531.9 MB/s, size: 469.7 KB)


...

WARNING Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  95%|█████████▍| 18/19 [00:03<00:00,  6.16it/s]

WARNING Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  5.94it/s]


                   all        593        109     0.0625     0.0183     0.0319    0.00796
                 human         52        109     0.0625     0.0183     0.0319    0.00796
Speed: 0.2ms preprocess, 4.8ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\train\yolov10n_bus_stop_finetuned8
✅ Модель дообучена и сохранена.


In [3]:
# Загружаем лучшую сохранённую модель
model = YOLO("runs/train/yolov10n_bus_stop_finetuned8/weights/best.pt")

# Предсказание на папке с картинками
results = model.predict(
    source="dataset/test/images",
    imgsz=640,
    conf=0.25,  # минимальная уверенность
    save=True,   # сохранить предсказания
    save_txt=True
)


image 1/1 c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\ai-core\dataset\test\images\20250809_164148.jpg: 384x640 (no detections), 56.1ms
Speed: 2.9ms preprocess, 56.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\predict
0 label saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\predict\labels


In [ ]:
metrics = model.val(
    data="dataset/data.yaml",
    imgsz=640,
    conf=0.25
)

# Подсчёт пассажиров в видеопотоке

In [ ]:
from ultralytics import YOLO

model = YOLO('best.pt')

# Запуск трекинга на видео
model.track(
    source='video.mp4',
    tracker='bytetrack.yaml',  # можно 'deepsort.yaml'
    conf=0.5,
    save=True
)


In [ ]:
# упрощённый фрагмент
if hasattr(res, 'masks') and res.masks is not None:
    im_with_masks = res.masks.render(res.orig_img)  # иногда работает напрямую
    frame = im_with_masks


In [ ]:
def dice_coef(gt_mask, pred_mask):
    gt = gt_mask.astype(bool)
    pr = pred_mask.astype(bool)
    inter = (gt & pr).sum()
    return 2*inter / (gt.sum() + pr.sum() + 1e-6)
